https://www.kaggle.com/competitions/ieee-fraud-detection/overview

In [15]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight

from xgboost import XGBClassifier

import lightgbm as lgb

from imblearn.over_sampling import SMOTE

import os

In [2]:
# Following datatype processing code comes from James Trotman's IEEE Fraud notebook: https://www.kaggle.com/code/jtrotman/ieee-fraud-adversarial-lgb-split-points
DTYPE = {
    'TransactionID': 'int32',
    'isFraud': 'int8',
    'TransactionDT': 'int32',
    'TransactionAmt': 'float32',
    'ProductCD': 'category',
    'card1': 'int16',
    'card2': 'float32',
    'card3': 'float32',
    'card4': 'category',
    'card5': 'float32',
    'card6': 'category',
    'addr1': 'float32',
    'addr2': 'float32',
    'dist1': 'float32',
    'dist2': 'float32',
    'P_emaildomain': 'category',
    'R_emaildomain': 'category',
}

IDX = 'TransactionID'
TGT = 'isFraud'

CCOLS = [f'C{i}' for i in range(1, 15)]
DCOLS = [f'D{i}' for i in range(1, 16)]
MCOLS = [f'M{i}' for i in range(1, 10)]
VCOLS = [f'V{i}' for i in range(1, 340)]

DTYPE.update((c, 'float32') for c in CCOLS)
DTYPE.update((c, 'float32') for c in DCOLS)
DTYPE.update((c, 'float32') for c in VCOLS)
DTYPE.update((c, 'category') for c in MCOLS)


DTYPE_ID = {
    'TransactionID': 'int32',
    'DeviceType': 'category',
    'DeviceInfo': 'category',
}

ID_COLS = [f'id_{i:02d}' for i in range(1, 39)]
ID_CATS = [
    'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
    'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38'
]

DTYPE_ID.update(((c, 'float32') for c in ID_COLS))
DTYPE_ID.update(((c, 'category') for c in ID_CATS))

In [3]:
# Import the data from CSV
this_dir = os.path.dirname(os.path.abspath('__file__'))
data_path = os.path.join(this_dir, 'fraud_data')
identity_data = pd.read_csv(data_path + '\\train_identity.csv', dtype=DTYPE_ID)
transaction_data = pd.read_csv(data_path + '\\train_transaction.csv', dtype=DTYPE)
data = transaction_data.merge(identity_data, how='inner', left_on='TransactionID', right_on='TransactionID')
print(data.shape)
data.head()

(144233, 434)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0,86506,50.000000,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,0,86535,15.000000,H,2803,100.0,150.0,visa,226.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,0,86549,75.887001,C,16496,352.0,117.0,mastercard,134.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,0,86555,16.495001,C,4461,375.0,185.0,mastercard,224.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0,86620,30.000000,H,1790,555.0,150.0,visa,226.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [4]:
# Check the imbalance. Fewer than 1% of transactions are fraudulent
data[['isFraud','TransactionID']].groupby('isFraud').count()

,TransactionID
isFraud,
0,132915
1,11318


In [5]:
# Check some distribution statistics
data.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,1.442330e+05,144233.000000,1.442330e+05,144233.000000,144233.000000,143331.000000,144061.000000,143277.000000,83786.000000,83786.000000,...,139369.00000,45113.000000,139318.000000,139261.000000,5159.000000,5169.000000,4747.000000,5132.000000,5163.000000,77586.000000
mean,3.236329e+06,0.078470,6.166958e+06,83.554527,9879.012528,391.325531,161.695511,191.166321,296.845032,86.198090,...,189.45137,14.237337,353.128143,403.882629,368.269806,16.002708,12.800927,329.608917,149.070312,26.508596
std,1.788496e+05,0.268911,4.807714e+06,99.850258,5047.643179,160.327850,19.410116,44.774338,99.307182,5.645163,...,30.37536,1.561301,141.095352,152.160324,198.847031,6.897665,2.372447,97.461090,32.101994,3.737502
min,2.987004e+06,0.000000,8.650600e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,100.00000,10.000000,100.000000,100.000000,100.000000,10.000000,11.000000,100.000000,100.000000,0.000000
25%,3.077142e+06,0.000000,1.885289e+06,25.452999,5713.000000,264.000000,150.000000,138.000000,204.000000,87.000000,...,166.00000,13.000000,266.000000,256.000000,252.000000,14.000000,11.000000,321.000000,119.000000,24.000000
50%,3.198818e+06,0.000000,4.913738e+06,50.000000,9633.000000,408.000000,150.000000,224.000000,299.000000,87.000000,...,166.00000,15.000000,341.000000,472.000000,252.000000,14.000000,11.000000,321.000000,149.000000,24.000000
75%,3.392923e+06,0.000000,1.025794e+07,100.000000,15063.000000,545.000000,185.000000,226.000000,330.000000,87.000000,...,225.00000,15.000000,427.000000,533.000000,486.500000,14.000000,15.000000,371.000000,169.000000,32.000000
max,3.577534e+06,1.000000,1.581103e+07,1800.000000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,229.00000,29.000000,671.000000,661.000000,854.000000,44.000000,26.000000,548.000000,216.000000,32.000000


In [6]:
# Fill categorical NaNs by adding 'missing' to categories first
for col in data.select_dtypes(include='category').columns:
    if 'missing' not in data[col].cat.categories:
        data[col] = data[col].cat.add_categories('missing')
    data[col] = data[col].fillna('missing')

# Fill numeric NaNs
for col in data.select_dtypes(include=['float32', 'int16', 'int32', 'float64']).columns:
    data[col] = data[col].fillna(-999)

# Encode categorical columns with .cat.codes
for col in data.select_dtypes(include='category').columns:
    data[col] = data[col].cat.codes

y = data[['isFraud']]
X = data.drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)

In [ ]:
# Handling imbalance using SMOTE
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

# Normalize numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Handle imbalance with SMOTE
sm = SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Train LightGBM
model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=64,
    colsample_bytree=0.8,
    subsample=0.9,
    random_state=0,
    class_weight='balanced',
    n_jobs=4
)

model.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_test, y_test)],
    eval_metric='auc'
)

# Predictions and evaluation
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC: {auc:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 106332, number of negative: 106332
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.598066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99670
[LightGBM] [Info] Number of data points in the train set: 212664, number of used features: 407
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



ROC AUC: 0.9706

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98     26583
           1       0.89      0.72      0.80      2264

    accuracy                           0.97     28847
   macro avg       0.93      0.86      0.89     28847
weighted avg       0.97      0.97      0.97     28847



In [14]:
# Handling imbalance using Downsampling from Majority
# Train-test split (on original, imbalanced data)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

# Combine X and y for training to downsample easily
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df['isFraud'] = y_train.values

# Downsample majority class in training set only
df_majority = train_df[train_df.isFraud == 0]
df_minority = train_df[train_df.isFraud == 1]

df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority) * 5,  # using 5x as many non-fraud, can play with this ratio
    random_state=42
)

df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Separate features and target
X_train = df_downsampled.drop('isFraud', axis=1)
y_train = df_downsampled['isFraud']

# Normalize numeric features (optional for trees)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train LightGBM
model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=64,
    colsample_bytree=0.8,
    subsample=0.9,
    random_state=0,
    class_weight='balanced',
    n_jobs=4
)

model.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_test, y_test)],
    eval_metric='auc'
)

# Predictions and evaluation
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC: {auc:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

C:\Users\jwesi\AppData\Local\Temp\ipykernel_17148\3920370883.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['isFraud'] = y_train.values
C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dty

[LightGBM] [Info] Number of positive: 106332, number of negative: 106332
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.676896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100039
[LightGBM] [Info] Number of data points in the train set: 212664, number of used features: 407
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\jwesi\Documents\GitHub\MovieLens-Data-Science\movielens_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



ROC AUC: 0.6634

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.01      0.02     26583
           1       0.08      1.00      0.15      2264

    accuracy                           0.09     28847
   macro avg       0.54      0.50      0.08     28847
weighted avg       0.93      0.09      0.03     28847



In [ ]:
# No sampling changes, only weight the model's learning
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

# Normalize numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Handle imbalance with SMOTE
sm = SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Train LightGBM
model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=64,
    colsample_bytree=0.8,
    subsample=0.9,
    random_state=0,
    class_weight='balanced',
    n_jobs=4
)

model.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_test, y_test)],
    eval_metric='auc'
)

# Predictions and evaluation
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC: {auc:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))